# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

设置embedding - zhipu

In [13]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from dotenv import load_dotenv, find_dotenv
import os
# 从环境变量中加载你的 API_KEY
_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../chroma-vmax-6'

# 创建嵌入模型
from langchain_community.embeddings import ZhipuAIEmbeddings

zhipu_embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key=zhipuai_api_key
)

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        collection_name="vmax-s",
        embedding_function=zhipu_embed
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

向量数据库已成功加载。


In [2]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：695


In [3]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：695


In [4]:
question = "安全加固的流程?"
docs = vectordb.similarity_search(question,k=15)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：15


打印一下检索到的内容

In [5]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 数据库数据安全加固体系数据库合规与漏洞参⻅第3.5章节
3.安全加固操作
3.1加固流程⾸次开局/新建⽹络、升级、周期性维护等的加固流程如下图所⽰。安全加固分为三个阶段：前期加固、现场加固、后期加固。
1)前期加固：主要是出⼚前的预加固，⽐如操作系统等，在初始版本的基础上保证将补丁安装⾄最新，合规配置符合基本要求等。
2)现场加固：现场安装配置的加固，由于现场客⼾有⾃⼰的⼯具扫描，会发现⼀些未符合客⼾要求的合规项，需要在现场进⾏加固。
3)后期加固：主要是后期扫描发现或者外部发现等需要处理的漏洞项。⽆论是哪个阶段的安全加固，都使⽤相同的安全加固⽅案。安全合规在如下阶段执⾏：
1)早期合规：镜像⽂件制作时完成所有的合规⼯作；
2)现场合规：由于前期未进⾏过合规，在现场操作；
3)漏洞发现：由于演进发现新的漏洞或合规，需要研发提供解决⽅案，并由现场操作解决。
3.2加固前的准备和检查
1、加固⼯具的获取：联系⽹服获取安全加固⼯具TECS-Inspector_SEC、⼯具使⽤指导书、⼀键安装指导书
2、通知客⼾以取得对应的许可或授权
3、加固前对数据进⾏安全备份
-----------------------------------------------------
检索到的第1个内容: 
 3.7加固后检查
3.7.1账⼾系统共安全加固检查安全加固指导
19-25
-----------------------------------------------------
检索到的第2个内容: 
 2、通知客⼾以取得对应的许可或授权
3、加固前对数据进⾏安全备份
3.3账⼾系统安全加固在系统初始配置时，部⻔、操作集、⻆⾊和⻆⾊集还没有创建。应当先创建部⻔、操作集、⻆⾊和⻆⾊集，再进⾏创建⽤⼾的操作。创建⽤⼾的流程参⻅表2-1。表2-1初始配置时⽤⼾创建流程步骤操作说明
1
2.1新建部⻔部⻔是对⾏政部⻔的⼀个模拟，便于对⽤⼾的组织和管理。维护⼈员可根据需要在部⻔管理中创建部⻔，并在不同的部⻔中创建⽤⼾。
2
（可选）2.2新建操作集将不同的操作权限组成⼀个集合，以集合的⽅式将操作权限分配给⻆⾊，降低权限管理的复杂性。
3
（可选）2.3新建⻆⾊分配给该⻆⾊对⽹管系统中各项资源的操作权限。
4
（可选）2.4新建⻆⾊集将不同的⻆⾊组成⼀个⻆⾊集，

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [6]:
from langchain_openai import ChatOpenAI
zhipuai_llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=zhipuai_api_key,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

## 3. 构建检索问答链

prompts

In [ ]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [ ]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(zhipuai_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


In [ ]:
question_1 = "什么是安全加固？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

### 创建一个基于模板的检索链： rerank检索版本

In [ ]:
# 测试API

import cohere
# co = cohere.ClientV2()
co = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
docs = [
    "Carson City is the capital city of the American state of Nevada.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.",
    "Capitalization or capitalisation in English grammar is the use of a capital letter at the start of a word. English usage varies from capitalization in other languages.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.",
    "Capital punishment has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
]
response = co.rerank(
    model="rerank-v3.5",
    query="What is the capital of the United States?",
    documents=docs,
    top_n=3,
)
print(response)



In [8]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索
# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")

compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


qa_chain = RetrievalQA.from_chain_type(
    zhipuai_llm,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)


C:\Users\will\AppData\Local\Temp\ipykernel_3604\1383225845.py:9: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank(


### 创建一个基于模板的检索链： BM25检索版本

## 4.检索问答链效果测试

In [ ]:
question_1 = "什么是安全加固？"
question_2 = "严威是谁？"

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [ ]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

In [ ]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

### 4.2 大模型自己回答的效果

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
zhipuai_llm.predict(prompt_template)

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
zhipuai_llm.predict(prompt_template)

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能-v2025

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [9]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

C:\Users\will\AppData\Local\Temp\ipykernel_3604\2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

v2024

In [ ]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    zhipuai_llm,
    retriever=retriever,
    memory=memory
)
question = "什么是安全加固？"
result = qa({"question": question})
print(result['answer'])

V2025

不带memory

In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索
# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
# compressor = CohereRerank(
#     client=cohere_client,
#     top_n=5  # 保留Top5相关文档
# )
compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# qa_chain = RetrievalQA.from_chain_type(zhipuai_llm,
#                                        retriever=base_retriever,
#                                        return_source_documents=True,
#                                        chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

qa_chain = RetrievalQA.from_chain_type(
    zhipuai_llm,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)

result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])


新增MEMORY

In [ ]:
from langchain.chains import ConversationalRetrievalChain

from langchain.prompts import PromptTemplate

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template="""
    你是一个专业的问答助手。请根据对话历史和提供的上下文回答问题。
    
    历史对话：
    {chat_history}
    
    上下文：
    {context}
    
    问题：{question}
    
    回答：
    """
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=zhipuai_llm,
    retriever=compression_retriever,
    memory=memory,
    # return_source_documents=True,
    output_key="answer",  # 明确指定存储到内存的键
    combine_docs_chain_kwargs={  # 替代chain_type_kwargs
        "prompt": QA_CHAIN_PROMPT
    },
    verbose=True, # 独立传递verbose参数
)


questions = [
    "什么是VMAX的安全加固？", 
    "安全加固的操作步骤？",  # 需记忆前一轮的"主要内容"
    "整理成中文表格"  # 需合并多轮信息
]

for question in questions:
    result = qa_chain({"question": question})
    print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    print(f"引用的来源：{result['source_documents'][0].metadata}")  # 显示来源文档
    print("对话历史：", memory.load_memory_variables({}))

## 5. 添加历史对话的记忆功能 - v2024

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

In [ ]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "什么是南瓜书？"
result = qa({"question": question})
print(result['answer'])

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
question = "南瓜书包含哪些内容？"
result = qa({"question": question})
print(result['answer'])

可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。